### IMPORT DEPENDENCIES AND LIBRARY METHODS AND CLASSES

In [1]:
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from models.MLP import MLP0, MLP1, MLP2
from util import seed_everything, \
    train, test, preprocess_dataset, MultiTransforms, \
    TransformPCA, SatelliteDataset
import argparse
from torch.utils.data import DataLoader


In [2]:
seed_everything(11)
IN_DIMS = 36
OUT_DIMS = 6  # label 6 has zero items so 6 classes only
DEVICE = "cuda:0"
ROOT_DIR = "dataset"

### DATASET CREATION, PREPROCESSING AND DATALOADERS 

In [3]:
train_dataset = SatelliteDataset("./dataset", train=True)
test_dataset = SatelliteDataset("./dataset", train=False)

# mean center and normalize
transform, target_transform = preprocess_dataset(train_dataset)

train_dataset.transform = transform
train_dataset.target_transform = target_transform

test_dataset.transform = transform
test_dataset.target_transform = target_transform

In [4]:
train_loader = DataLoader(train_dataset, batch_size=1,
                          shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=256,
                         shuffle=False, num_workers=2)

In [ ]:
def fit(
    model: nn.Module,
    train_loader,
    test_loader,
    args,
):
    train_loss = []
    train_accuracies = []

    test_loss = []
    test_accuracies = []

    early_stop_count = 0
    early_stop_patience = 5
    early_stop_tol = 1e-3

    best_train_acc = 0
    best_test_acc = 0

    progress_bar = tqdm(range(args.max_epochs),
                        desc="Epochs", disable=not args.verbose)

    for i in progress_bar:
        losses, train_acc = train(model, train_loader,
                                  lr=args.lr, device=args.device)
        loss = losses[-1]
        print(loss)
        train_loss.append(loss)
        train_accuracies.append(train_acc)
        best_train_acc = max(best_train_acc, train_acc)

        test_acc = 0
        progress_dict = {"Loss": loss, "Train Acc": train_acc}

        if i % args.test_freq == 0:
            results = test(model, test_loader,
                           device=args.device, metrics=["Accuracy", "Loss"])
            test_acc = results["Accuracy"]
            test_accuracies.append(test_acc)
            test_loss.append(results["Loss"])
            progress_dict["Test Accuracy"] = test_acc
            best_test_acc = max(best_test_acc, test_acc)

        if i % args.log_freq == 0 and args.verbose:
            progress_bar.set_postfix(progress_dict)

        if len(train_loss) > 1 and abs(loss-train_loss[-2]) < early_stop_tol:
            early_stop_count += 1
            if (early_stop_count >= early_stop_patience):
                print(f"Early stopping after {i} epochs.")
                break
        else:
            early_stop_count = 0

    if args.plot is True:
        # plot loss and accuracy in same plot
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        ax[0].plot(train_loss, label="train")
        ax[0].plot(test_loss, label="test")
        ax[0].set_xlabel("Epoch")
        ax[0].set_ylabel("Loss")

        ax[1].plot(train_accuracies, label="train")
        ax[1].plot(test_accuracies, label="test")
        ax[1].set_xlabel("Epoch")
        ax[1].set_ylabel("Accuracy")

        ax[0].legend()
        ax[1].legend()
        plt.show()

    return best_train_acc, best_test_acc


## Results for model architecture:
1. 0 hidden layers
2. sigmoid activation function
3. input_dim = 36, output_dim = 6

In [6]:
models = {
    "MLP0": MLP0(IN_DIMS, OUT_DIMS),
    "MLP1_0": MLP1(IN_DIMS, OUT_DIMS, 2),
    "MLP1_1": MLP1(IN_DIMS, OUT_DIMS, 6),
    "MLP2_0": MLP2(IN_DIMS, OUT_DIMS, 2, 3),
    "MLP2_1": MLP2(IN_DIMS, OUT_DIMS, 3, 2),
}

results = dict()
lrs = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]

for name, model in models.items():
    model = model.to(DEVICE)
    results[name] = []
    for lr in lrs:
        args = argparse.Namespace(max_epochs=5, lr=lr, device=DEVICE,
                                  verbose=False, log_freq=1, test_freq=1, plot=False)
        train_acc,  test_acc = fit(model, train_loader, test_loader, args)
        results[name].append((train_acc, test_acc))
        print(f"{name} with lr {lr} test accuracy: {test_acc}")

MLP0 with lr 0.1 test accuracy: 0.8434217108554277
MLP0 with lr 0.01 test accuracy: 0.8264132066033016
MLP0 with lr 0.001 test accuracy: 0.817408704352176
MLP0 with lr 0.0001 test accuracy: 0.817408704352176
MLP0 with lr 1e-05 test accuracy: 0.8169084542271136
MLP1_0 with lr 0.1 test accuracy: 0.3751875937968984
MLP1_0 with lr 0.01 test accuracy: 0.3371685842921461
MLP1_0 with lr 0.001 test accuracy: 0.23861930965482742
MLP1_0 with lr 0.0001 test accuracy: 0.23761880940470234
MLP1_0 with lr 1e-05 test accuracy: 0.23311655827913957


KeyboardInterrupt: 

In [ ]:
pca = PCA(n_components=2)
pca.fit(train_dataset.data)
pca_transform = TransformPCA(pca)
transforms = MultiTransforms([mtransform, pca_transform])

train_dataset.transform = transforms
train_dataset.target_transform = mtarget_transform
